## Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice.



### Q1. Running Elastic

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

``` curl localhost:9200 ```

What's the ```version.build_hash``` value?

In [3]:
!curl localhost:9200 #"42f05b9372a9a4a470db3b52817899b99a76ee73"

{
  "name" : "ec1bcb4f6551",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "ZxzLghYcRfK4R7r61bmjIA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   539  100   539    0     0   5861      0 --:--:-- --:--:-- --:--:--  5923


### Getting the data
Now let's get the FAQ data. You can run this snippet:

In [6]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

``` pip install elasticsearch ```

Which function do you use for adding your data to elastic?

In [8]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

### Q3. Searching

Now let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

In [11]:
from tqdm.auto import tqdm

c:\Users\daniel\Documents\Universidad\Cursos\LLMZoomCamp\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [01:00<00:00, 15.69it/s]


In [13]:
query = "How do I execute a command in a running docker container?"

In [18]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
    


In [22]:
response['hits']['hits'][0] #84.050095

{'_index': 'course-questions',
 '_id': 'wv78RpABYgsTHHJGNtOa',
 '_score': 84.050095,
 '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'}}

### Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

- How do I debug a docker container?
- How do I copy files from a different folder into docker container’s working directory?
- How do Lambda container images work?
- How can I annotate a graph?


In [23]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [24]:
elastic_search(query) #'How do I copy files from a different folder into docker container’s working directory?'

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

### Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

In [25]:
results = elastic_search(query)

In [35]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_entries = []
    for doc in search_results:
        context_entry = f"section: {doc['section']}\n\nquestion: {doc['question']}\n\nanswer: {doc['text']}\n"
        context_entries.append(context_entry)
    
    context = "\n\n".join(context_entries)

    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [36]:
len(build_prompt("How do I execute a command in a running docker container?", results))

1644

### Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

``` pip install tiktoken ```

Let's calculate the number of tokens in our query:

``` encoding = tiktoken.encoding_for_model("gpt-4o") ```

Use the encode function. How many tokens does our prompt have?

In [37]:
!pip install tiktoken

     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/42.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 503.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/798.9 kB ? eta -:--:--
   --- ------------------------------------ 61.4/798.9 kB 3.2 MB/s eta 0:00:01
   ------ --------------------------------- 122.9/798.9 kB 1.8 MB/s eta 0:00:01
   ------------ --------------------------- 245.8/798.9 kB 1.9 MB/s eta 0:00:01
   ------------------------ --------------- 491.5/798.9 kB 2.6 MB/s eta 0:00:01
   ---------------------------------- ----- 696.3/798.9 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 798.9/798.9 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/269.0 kB ? eta -:--:--
   ---------------------------------------  266.2/269.0 kB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 269.0/269.0 kB 4.2 MB/s

In [39]:
import tiktoken

In [40]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [45]:
len(encoding.encode(build_prompt("How do I execute a command in a running docker container?", results)))

355

In [44]:
encoding.decode_single_token_bytes(63842)

b"You're"